In [ ]:
!pip3 install tmdbv3api

In [ ]:
import re
import requests
import numpy as np
import pandas as pd
from tmdbv3api import TMDb, Movie

In [ ]:
# Initialize TMDb API
tmdb = TMDb()
tmdb.api_key = "feb953d34c2f897d2132e7951819f404"  # Set your API key
tmdb_movie = Movie()

In [ ]:
review_df = pd.read_csv("./reviews.txt", sep="\t", names=["Reviews", "Comments"])

In [ ]:
review_df

In [ ]:
movie_df = pd.read_csv("../datasets/final_data.csv")

In [ ]:
movie_df

In [ ]:
def sentiment_analysis(movie_name, casts, reviews=review_df):
    sentiment_dict = {}
    pos, neg = 0, 0

    movie_name = re.sub(r"[^\w\s]", "", movie_name).strip().split()
    cast_tokens = {cast: cast.lower().split() for cast in casts}

    for _, review in reviews.iterrows():
        review_content = review["Comments"]

        for cast, tokens in cast_tokens.items():
            movie_presence = all(
                re.search(rf"\b{word}\b", review_content, re.IGNORECASE)
                for word in movie_name
            )
            cast_presence = any(
                re.search(rf"\b{word}\b", review_content, re.IGNORECASE)
                for word in tokens
            )

            if movie_presence and cast_presence:
                if review["Reviews"] == 1:
                    pos += 1
                elif review["Reviews"] == 0:
                    neg += 1

                total = pos + neg
                sentiment_score = pos / total if total > 0 else 0.5
                sentiment_dict[cast] = sentiment_score

    return sentiment_dict

In [ ]:
for index, rows in movie_df.iterrows():
    movie_name = rows["movie_title"]
    casts = rows[
        ["director_name", "actor_1_name", "actor_2_name", "actor_3_name"]
    ].tolist()
    casts = [re.sub(r"[^\w\s]", "", cast).strip() for cast in casts]
    print(index)
    cast_sentiment = sentiment_analysis(movie_name, casts)
    movie_df.at[index, "cast_sentiment"] = cast_sentiment

In [ ]:
movie_df[(movie_df["cast_sentiment"].notna()) & (movie_df["cast_sentiment"] != {})]

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb,cast_sentiment
170,Steven Spielberg,Tom Cruise,Lisa Ann Walter,Rick Gonzalez,Adventure Sci-Fi Thriller,war of the worlds,Tom Cruise Lisa Ann Walter Rick Gonzalez Steve...,{'Tom Cruise': 0.0}
189,Ron Howard,Tom Hanks,Seth Gabel,Jürgen Prochnow,Mystery Thriller,the da vinci code,Tom Hanks Seth Gabel Jürgen Prochnow Ron Howar...,{'Tom Hanks': 0.0}
397,Brian De Palma,Tom Cruise,Kristin Scott Thomas,Vanessa Redgrave,Action Adventure Thriller,mission: impossible,Tom Cruise Kristin Scott Thomas Vanessa Redgra...,{'Tom Cruise': 0.9117647058823529}
717,David O. Russell,Jennifer Lawrence,Robert De Niro,Bradley Cooper,Biography Comedy Drama,joy,Jennifer Lawrence Robert De Niro Bradley Coope...,{'David O. Russell': 0.0}
952,John McTiernan,Connie Nielsen,Harry Connick Jr.,Tim Daly,Action Crime Drama Mystery Thriller,basic,Connie Nielsen Harry Connick Jr. Tim Daly John...,{'Harry Connick Jr.': 0.0}
1145,Etan Cohen,Will Ferrell,Alison Brie,Craig T. Nelson,Comedy Crime,get hard,Will Ferrell Alison Brie Craig T. Nelson Etan ...,"{'Will Ferrell': 1.0, 'Craig T. Nelson': 1.0}"
2295,Penny Marshall,Tom Hanks,Jon Lovitz,John Heard,Comedy Drama Family Fantasy Romance,big,Tom Hanks Jon Lovitz John Heard Penny Marshall...,{'Tom Hanks': 1.0}
2705,Ang Lee,Jake Gyllenhaal,Heath Ledger,Anne Hathaway,Drama Romance,brokeback mountain,Jake Gyllenhaal Heath Ledger Anne Hathaway Ang...,"{'Jake Gyllenhaal': 1.0, 'Heath Ledger': 1.0, ..."
3159,Malcolm D. Lee,Harold Perrineau,Sanaa Lathan,Jarrod Bunch,Comedy Drama,the best man,Harold Perrineau Sanaa Lathan Jarrod Bunch Mal...,{'Malcolm D. Lee': 0.5}
3326,Paul Haggis,Don Cheadle,Loretta Devine,Jennifer Esposito,Crime Drama Thriller,crash,Don Cheadle Loretta Devine Jennifer Esposito P...,{'Don Cheadle': 1.0}
